This Code will merge the companys names in the n2225 with their adresses from yahoo finance. Also it will convert the adress to longitude and latitude with geocoder.

In [2]:
import pandas as pd 

import yfinance as yf 

import geocoder

In [6]:


# Read CSV File and start wirth line two as Header
nikkei_companies = pd.read_csv("Users/jan/Techlabs_WS_Team3/01_data/03_additional/01_N225_Japan/N225.csv")


In [7]:
nikkei_companies.head()

,Ticker,Company_name,Industry
0,4151.T,KYOWA KIRIN CO.. LTD.,Pharmaceuticals
1,4502.T,TAKEDA PHARMACEUTICAL CO.. LTD.,Pharmaceuticals
2,4503.T,ASTELLAS PHARMA INC.,Pharmaceuticals
3,4506.T,SUMITOMO PHARMA CO.. LTD.,Pharmaceuticals
4,4507.T,SHIONOGI & CO.. LTD.,Pharmaceuticals


In [8]:
# Function to return adress from yahoo finance based on the ticker number 
def get_address_and_city(ticker):
    try:
        company = yf.Ticker(ticker)
        address = company.info.get('address1', 'adress not available')
        city = company.info.get('city', 'city not available')
        return address, city
    except Exception as e:
        print(f"error by loading the data for {ticker}: {e}")
        return 'error by loading the data', 'error by loading the data '


# add new columns "Adress" & "City" to dataframe 
nikkei_companies[['Address', 'City']] = nikkei_companies['Ticker'].apply(lambda ticker: pd.Series(get_address_and_city(ticker)))



We want to display the companies on the Map: For the Visualization we need the Longitude and Latitude of the City from each Company - With Geocoder we can translate the City Longitude and Latitude data. We created an Account with Mapbox to use the services via a passkey.

In [9]:
#Function to transform city into latitude and longitude - add new columns to the dateframe (longitude and latitude)

def get_lat_lon(City):
    try:
        g = geocoder.mapbox(City, key="pk.eyJ1IjoidGVjaGxhYnMzIiwiYSI6ImNtNzBib2xyczAwZHoycnBiM2hxZ24zcngifQ.AaSHNEfc-cnR4uDdEO4gsw")  # Mapbox Geocoding
        if g.ok:
            return g.latlng[0], g.latlng[1]
        return None, None
    except Exception as e:
        print(f"Fehler bei {address}: {e}")
        return None, None


nikkei_companies["latitude"], nikkei_companies["longitude"] = zip(*[get_lat_lon(addr) for addr in nikkei_companies["City"]])



In [10]:
#Save it as pickle to use it in the dashboard
nikkei_companies.to_pickle("Users/jan/Techlabs_WS_Team3/02_code/0_preprocess/nikkei_companies.pkl")